In [1]:
import pandas as pd
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.compat import lzip

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
sales_train_evaluation = pd.read_csv('sales_train_evaluation.csv')

In [4]:
sales_train_validation = pd.read_csv('sales_train_validation.csv')

In [5]:
sell_prices=pd.read_csv('sell_prices.csv')

In [6]:
sell_prices.head

<bound method NDFrame.head of         store_id        item_id  wm_yr_wk  sell_price
0           CA_1  HOBBIES_1_001     11325        9.58
1           CA_1  HOBBIES_1_001     11326        9.58
2           CA_1  HOBBIES_1_001     11327        8.26
3           CA_1  HOBBIES_1_001     11328        8.26
4           CA_1  HOBBIES_1_001     11329        8.26
...          ...            ...       ...         ...
6841116     WI_3    FOODS_3_827     11617        1.00
6841117     WI_3    FOODS_3_827     11618        1.00
6841118     WI_3    FOODS_3_827     11619        1.00
6841119     WI_3    FOODS_3_827     11620        1.00
6841120     WI_3    FOODS_3_827     11621        1.00

[6841121 rows x 4 columns]>

In [7]:
sales_train_validation.head

<bound method NDFrame.head of                                   id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_validation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_validation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_validation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_validation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_validation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1

In [8]:


sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [136]:
df=pd.read_csv('sales_train_validation.csv')

In [137]:
data=pd.read_csv('sales_train_validation.csv')
data

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [81]:
def cal_days(data): #calculate and output a list of days 
    start_day=1
    day_format="d_"
    cols=[]
    for j in range(math.ceil((len(data.columns)-6)//7) ):
        for i in range(1,8):
            cols.append(day_format+str(start_day))
            start_day=start_day+1
    return cols


    

In [138]:
cols=cal_days(df)
cols[-10:]

['d_1902',
 'd_1903',
 'd_1904',
 'd_1905',
 'd_1906',
 'd_1907',
 'd_1908',
 'd_1909',
 'd_1910',
 'd_1911']

In [13]:
len(cols)

1911

In [83]:
def create_week(cols,df): # create columns for each week and quantity of products sold that week
    for i in range(0,len(cols),7):
        temp=cols[i:i+7]

        df["week" +str(int(i/7+1))]=df[temp].sum(axis=1)
    

In [139]:
create_week(cols,data)


In [140]:
data

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,week264,week265,week266,week267,week268,week269,week270,week271,week272,week273
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,6,8,10,6,6,10,3,10,5,9
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,3,1,2,3,0,0,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,1,0,10,3,1,1,2,5,7
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,18,7,10,14,16,15,10,15,14
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,12,7,6,8,9,10,9,7,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,4,1,3,5,0,0,0,0,4,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,4,4,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,4,2,11,12,9,4,2,9,10,4
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,12,0,8,4,10,7,9,7,7,5


In [16]:
def cal_extra_days(data):# calcualte extra days not included in weeks listed
    if not ((len(data.columns)-6)/7).is_integer():
        extra_days=(len(data.columns)-6)-(((len(data.columns)-6)//7)*7)
    return extra_days

In [141]:
extra_days=cal_extra_days(data)
extra_days

2

In [18]:
def add_extra_days_col(data,cols): # add extra days to list to sum all later to week
    temp_col=[]
    extra_days=cal_extra_days(data) 
    for i in range(((len(cols))//7)*7+1,((len(cols))//7)*7+extra_days+1):
        temp="d_"+str(i)
        temp_col.append(temp)   
    
    data["week" +str((len(cols))//7+1)]=data[temp_col].sum(axis=1)

In [142]:
add_extra_days_col(data,cols)
data

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,week265,week266,week267,week268,week269,week270,week271,week272,week273,week274
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,10,6,6,10,3,10,5,9,2
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,3,1,2,3,0,0,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,10,3,1,1,2,5,7,2
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18,7,10,14,16,15,10,15,14,9
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,7,6,8,9,10,9,7,8,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,3,5,0,0,0,0,4,1,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,4,4,0,1
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,11,12,9,4,2,9,10,4,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,8,4,10,7,9,7,7,5,4


In [92]:
data

0         0
1         0
2         0
3         0
4         0
         ..
30485     8
30486     5
30487    15
30488     0
30489     0
Name: week1, Length: 30490, dtype: int64

In [186]:
def create_week_sales_info(df,cols):
    info_sales=df.iloc[:,:6]
    week_sales=df.iloc[:,-math.ceil((len(cols))/7)-1:]
    result_df = pd.concat([info_sales, week_sales], axis=1)
    return result_df

In [222]:
result_df=create_week_sales_info(data,cols)
result_df

,id,item_id,dept_id,cat_id,store_id,state_id,week1,week2,week3,week4,...,week265,week266,week267,week268,week269,week270,week271,week272,week273,week274
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,10,6,6,10,3,10,5,9,2
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,3,1,2,3,0,0,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,10,3,1,1,2,5,7,2
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18,7,10,14,16,15,10,15,14,9
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,7,6,8,9,10,9,7,8,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,8,16,10,41,...,1,3,5,0,0,0,0,4,1,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,5,11,7,6,...,0,0,0,0,0,0,4,4,0,1
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,15,35,55,24,...,2,11,12,9,4,2,9,10,4,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,8,4,10,7,9,7,7,5,4


In [224]:
result_df

,id,item_id,dept_id,cat_id,store_id,state_id,week1,week2,week3,week4,...,week265,week266,week267,week268,week269,week270,week271,week272,week273,week274
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,10,6,6,10,3,10,5,9,2
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,3,1,2,3,0,0,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,10,3,1,1,2,5,7,2
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18,7,10,14,16,15,10,15,14,9
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,7,6,8,9,10,9,7,8,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,8,16,10,41,...,1,3,5,0,0,0,0,4,1,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,5,11,7,6,...,0,0,0,0,0,0,4,4,0,1
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,15,35,55,24,...,2,11,12,9,4,2,9,10,4,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,8,4,10,7,9,7,7,5,4


In [24]:
def create_data_predict(result,sell_data):
    start_week="week"+str((int(str(sell_data.iloc[0,2])[2])*52-52+int(str(sell_data.iloc[0,2])[3:])))
    x=result_df.loc[1][start_week:].values.tolist()
    y=sell_data.iloc[0:146,3].values.tolist()

    return x,y

In [607]:
x,y=create_data_predict(result_df,sell_prices)

In [608]:
len(x)

146

In [609]:
len(y)

146

In [602]:
x=result_df.loc[1][start_week:]

In [603]:
y=sell_prices.iloc[0:146,3]

In [610]:
y

[9.58,
 9.58,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,

In [604]:
data = pd.concat([x,y], axis=1,ignore_index=False)

In [611]:
x= sm.add_constant(x)

In [ ]:
model = sm.OLS(y,x)

In [ ]:
results = model.fit()

In [ ]:
results.params

In [ ]:
results.tvalues

In [ ]:
print(results.t_test([1, 0]))

In [ ]:
model=sm.OLS(y, x).fit()

In [612]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                   0.02564
Date:                Wed, 13 Mar 2024   Prob (F-statistic):              0.873
Time:                        18:45:18   Log-Likelihood:                 65.905
No. Observations:                 146   AIC:                            -127.8
Df Residuals:                     144   BIC:                            -121.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.2825      0.018    469.362      0.000       8.248       8.317
x1            -0.0010      0.006     -0.160      0.873      -0.013       0.011
==============================================================================
Omnibus:                      256.038   Durbin-Watson:                   0.509
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28398.698
Skew:                           8.221   Prob(JB):                         0.00
Kurtosis:                      69.317   Cond. No.                         4.27
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
plt.rc("figure", figsize=(16, 8))


In [ ]:
plt.rc("font", size=14)

In [ ]:
import seaborn as sns

In [ ]:
p=model.params

In [ ]:
p[0]

In [ ]:
data

In [ ]:
data.rename(columns={1:"quantity"},inplace=True)

In [ ]:
from linearmodels import IV2SLS 

In [ ]:
iv_reg = IV2SLS.from_formula("sell_price ~ quantity", data).fit()


In [ ]:
iv_reg.summary

In [ ]:
test=sell_prices.loc[0:146]
test[]

In [ ]:
price_range=sell_prices.groupby(['store_id','item_id'])['sell_price'].apply(lambda x: max(x) - min(x))
max_change_product=price_range.idxmax()
max_change_value=price_range.max()

In [ ]:
max_change_product

In [ ]:
price_range.max()

In [ ]:
max_change_value

In [ ]:
test=sell_prices.loc[sell_prices["store_id"]=="WI_3" ]

In [ ]:
test=test.loc[sell_prices["item_id"]=="HOUSEHOLD_2_406"]

In [ ]:
test

In [ ]:
display(test.to_string())

In [ ]:
dates=test.loc[test["sell_price"] >13]["wm_yr_wk"]

In [ ]:
start_week="week"+str((int(str(sell_prices.iloc[0,2])[2])*52-52+int(str(sell_prices.iloc[0,2])[3:])))
x=result_df.loc[1][start_week:].values.tolist()
y=sell_prices.iloc[0:146,3].values.tolist()

In [ ]:
start_week=dates.iloc[0]


In [ ]:
start_week
x=result_df.loc[1][start_week:].values.tolist()


In [ ]:
result=result_df.loc[result_df["item_id"]=="HOUSEHOLD_2_406"]

In [ ]:
result=result.loc[result["store_id"]=="WI_3"]

In [ ]:
x=result.iloc[:,6:].values.tolist()

In [ ]:
x=x[0]

In [ ]:
x[115:125]

In [ ]:
y=sell_prices.iloc[0:146,3].values.tolist()

In [ ]:
test=test["sell_price"].values.tolist()

In [ ]:
test[121]

In [ ]:
x= sm.add_constant(x)

In [ ]:
sm.OLS(test[0:-8],x).fit().summary()

In [ ]:
#Demand Forecasting Attempt

In [29]:
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [58]:
line=result_df.loc[0].T
line=line[132:]
line

week128     0
week129     1
week130     0
week131     2
week132     2
           ..
week270     3
week271    10
week272     5
week273     9
week274     2
Name: 0, Length: 147, dtype: object

In [61]:
week=[]
for i in range(128,275):
    week.append(i)
len(week)
week

[128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274]

In [62]:
fig_demand = px.line( x=week,
                     y=line,
                     title='Demand Over Time')
fig_demand.show()

In [63]:

time_series = data.set_index(week)[line]

differenced_series = time_series.diff().dropna()

# Plot ACF and PACF of differenced time series
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_acf(differenced_series, ax=axes[0])
plot_pacf(differenced_series, ax=axes[1])
plt.show()

KeyError: 'None of [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274] are in the columns'

In [64]:
time_series = data.set_index(week)[line]

KeyError: 'None of [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274] are in the columns'

In [65]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 2) #2 because the data contains a time period of 2 months only
model = SARIMAX(y, order=order, seasonal_order=seasonal_order)
model_fit = model.fit(disp=False)

C:\Users\lywt9\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [68]:
future_steps = 10
predictions = model_fit.predict(len(y), len(y) + future_steps - 1)
predictions = predictions.astype(int)
print(predictions)

[8 8 8 8 8 8 8 8 8 8]


In [282]:
new=result_df.copy()

In [283]:
new

,id,item_id,dept_id,cat_id,store_id,state_id,week1,week2,week3,week4,...,week265,week266,week267,week268,week269,week270,week271,week272,week273,week274
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,10,6,6,10,3,10,5,9,2
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,3,1,2,3,0,0,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,10,3,1,1,2,5,7,2
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18,7,10,14,16,15,10,15,14,9
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,7,6,8,9,10,9,7,8,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,8,16,10,41,...,1,3,5,0,0,0,0,4,1,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,5,11,7,6,...,0,0,0,0,0,0,4,4,0,1
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,15,35,55,24,...,2,11,12,9,4,2,9,10,4,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,8,4,10,7,9,7,7,5,4


In [284]:
def cal_weeks(data): #calculate and output a list of days 
    start_day=1
    day_format="week"
    cols=[]
    print(len(data.columns))
    for j in range(1,math.ceil((len(data.columns)-6)) ):
            cols.append(day_format+str(j))
            
    return cols
weeks=cal_weeks(new)

280


In [285]:
weeks

['week1',
 'week2',
 'week3',
 'week4',
 'week5',
 'week6',
 'week7',
 'week8',
 'week9',
 'week10',
 'week11',
 'week12',
 'week13',
 'week14',
 'week15',
 'week16',
 'week17',
 'week18',
 'week19',
 'week20',
 'week21',
 'week22',
 'week23',
 'week24',
 'week25',
 'week26',
 'week27',
 'week28',
 'week29',
 'week30',
 'week31',
 'week32',
 'week33',
 'week34',
 'week35',
 'week36',
 'week37',
 'week38',
 'week39',
 'week40',
 'week41',
 'week42',
 'week43',
 'week44',
 'week45',
 'week46',
 'week47',
 'week48',
 'week49',
 'week50',
 'week51',
 'week52',
 'week53',
 'week54',
 'week55',
 'week56',
 'week57',
 'week58',
 'week59',
 'week60',
 'week61',
 'week62',
 'week63',
 'week64',
 'week65',
 'week66',
 'week67',
 'week68',
 'week69',
 'week70',
 'week71',
 'week72',
 'week73',
 'week74',
 'week75',
 'week76',
 'week77',
 'week78',
 'week79',
 'week80',
 'week81',
 'week82',
 'week83',
 'week84',
 'week85',
 'week86',
 'week87',
 'week88',
 'week89',
 'week90',
 'week91',
 'week92

In [286]:
len(weeks)//4

68

In [287]:
#Month Hierachy
def create_month(cols,df): # create columns for each week and quantity of products sold that week
    for i in range(0,len(cols),4):
        temp=cols[i:i+4]
        print(temp)
        
        
        df["month" +str(i//4)]=df[temp].sum(axis=1)
create_month(weeks,new)

['week1', 'week2', 'week3', 'week4']
['week5', 'week6', 'week7', 'week8']
['week9', 'week10', 'week11', 'week12']
['week13', 'week14', 'week15', 'week16']
['week17', 'week18', 'week19', 'week20']
['week21', 'week22', 'week23', 'week24']
['week25', 'week26', 'week27', 'week28']
['week29', 'week30', 'week31', 'week32']
['week33', 'week34', 'week35', 'week36']
['week37', 'week38', 'week39', 'week40']
['week41', 'week42', 'week43', 'week44']
['week45', 'week46', 'week47', 'week48']
['week49', 'week50', 'week51', 'week52']
['week53', 'week54', 'week55', 'week56']
['week57', 'week58', 'week59', 'week60']
['week61', 'week62', 'week63', 'week64']
['week65', 'week66', 'week67', 'week68']
['week69', 'week70', 'week71', 'week72']
['week73', 'week74', 'week75', 'week76']
['week77', 'week78', 'week79', 'week80']
['week81', 'week82', 'week83', 'week84']
['week85', 'week86', 'week87', 'week88']
['week89', 'week90', 'week91', 'week92']
['week93', 'week94', 'week95', 'week96']
['week97', 'week98', 'wee

In [288]:
new

,id,item_id,dept_id,cat_id,store_id,state_id,week1,week2,week3,week4,...,month59,month60,month61,month62,month63,month64,month65,month66,month67,month68
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,22,18,16,12,21,26,19,30,28,9
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,15,9,9,11,27,4,4,7,5,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,9,21,13,20,42,13,9,14,9,7
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,52,50,58,46,45,49,64,49,56,14
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,39,38,38,36,45,15,32,30,34,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,8,16,10,41,...,17,18,15,21,27,22,25,9,4,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,5,11,7,6,...,0,0,0,0,0,0,0,0,8,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,15,35,55,24,...,13,5,3,8,23,23,19,34,25,4
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,23,47,43,58,41,41,34,22,30,5


In [291]:
def create_month_sales_info(df,cols):
    info_sales=df.iloc[:,:6]
    week_sales=df.iloc[:,-math.ceil((len(cols))/4)+1:]
    result_df = pd.concat([info_sales, week_sales], axis=1)
    return result_df

In [292]:
monthly_sale=create_month_sales_info(new,weeks)
monthly_sale

,id,item_id,dept_id,cat_id,store_id,state_id,month1,month2,month3,month4,...,month59,month60,month61,month62,month63,month64,month65,month66,month67,month68
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,22,18,16,12,21,26,19,30,28,9
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,15,9,9,11,27,4,4,7,5,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,9,21,13,20,42,13,9,14,9,7
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,10,21,33,25,...,52,50,58,46,45,49,64,49,56,14
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,50,...,39,38,38,36,45,15,32,30,34,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,112,104,62,33,...,17,18,15,21,27,22,25,9,4,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,40,24,25,11,...,0,0,0,0,0,0,0,0,8,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,87,37,31,28,...,13,5,3,8,23,23,19,34,25,4
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,23,47,43,58,41,41,34,22,30,5


In [351]:
monthly_sale.loc[(monthly_sale['store_id'] == "CA_1") & (monthly_sale['item_id'] == "HOBBIES_1_001")]

TypeError: '(slice(None, None, None), slice(6, None, None))' is an invalid key

In [300]:
test=sell_prices.loc[(monthly_sale['store_id'] == "CA_1") & (sell_prices['item_id'] == "HOBBIES_1_001")]

In [321]:
test

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38
150,CA_1,HOBBIES_1_001,11618,8.38
151,CA_1,HOBBIES_1_001,11619,8.38
152,CA_1,HOBBIES_1_001,11620,8.38


In [307]:
month=[]
for i in range(1,69):
    month.append("month"+str(i))
month

['month1',
 'month2',
 'month3',
 'month4',
 'month5',
 'month6',
 'month7',
 'month8',
 'month9',
 'month10',
 'month11',
 'month12',
 'month13',
 'month14',
 'month15',
 'month16',
 'month17',
 'month18',
 'month19',
 'month20',
 'month21',
 'month22',
 'month23',
 'month24',
 'month25',
 'month26',
 'month27',
 'month28',
 'month29',
 'month30',
 'month31',
 'month32',
 'month33',
 'month34',
 'month35',
 'month36',
 'month37',
 'month38',
 'month39',
 'month40',
 'month41',
 'month42',
 'month43',
 'month44',
 'month45',
 'month46',
 'month47',
 'month48',
 'month49',
 'month50',
 'month51',
 'month52',
 'month53',
 'month54',
 'month55',
 'month56',
 'month57',
 'month58',
 'month59',
 'month60',
 'month61',
 'month62',
 'month63',
 'month64',
 'month65',
 'month66',
 'month67',
 'month68']

In [330]:
test.loc[0]["sell_price"]

9.58

In [315]:
start_week=(int(str(test.loc[0]["wm_yr_wk"])[2])-1)*52+(int(str(test.loc[0]["wm_yr_wk"])[3:]))
start_week

129

In [319]:
start_month=math.ceil(start_week/4)

In [320]:
start_month

33

In [328]:
y=[0]*68




In [333]:
test

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38
150,CA_1,HOBBIES_1_001,11618,8.38
151,CA_1,HOBBIES_1_001,11619,8.38
152,CA_1,HOBBIES_1_001,11620,8.38


In [347]:
counter=0
for i in range(start_month,68):
    y[i]=(test.loc[counter]["sell_price"]+test.loc[counter+1]["sell_price"]+test.loc[counter+2]["sell_price"]+test.loc[counter+3]["sell_price"])/4
    print(start_month)
    counter+=4

33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33


In [414]:
y=y[31:]

In [421]:
x=monthly_sale.loc[(monthly_sale['store_id'] == "CA_1") & (monthly_sale['item_id'] == "HOBBIES_1_001")].iloc[:,6:].values.tolist()[0]

In [422]:
x=x[31:]

In [424]:
x= sm.add_constant(x)

In [423]:
x

[5,
 12,
 14,
 12,
 19,
 22,
 10,
 12,
 10,
 10,
 7,
 12,
 18,
 14,
 15,
 18,
 15,
 19,
 18,
 20,
 19,
 20,
 24,
 12,
 2,
 18,
 20,
 22,
 18,
 16,
 12,
 21,
 26,
 19,
 30,
 28,
 9]

In [425]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     3.386
Date:                Wed, 13 Mar 2024   Prob (F-statistic):             0.0743
Time:                        11:52:26   Log-Likelihood:                -74.053
No. Observations:                  37   AIC:                             152.1
Df Residuals:                      35   BIC:                             155.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.3495      0.860      7.380      0.000       4.603       8.096
x1             0.0917      0.050      1.840      0.074      -0.009       0.193
==============================================================================
Omnibus:                       52.045   Durbin-Watson:                   0.758
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              249.527
Skew:                          -3.357   Prob(JB):                     6.55e-55
Kurtosis:                      13.806   Cond. No.                         49.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# 1 product month 1 location

In [ ]:
# Implementation of 1 product month, same state

In [637]:
same_state_sales=monthly_sale.loc[((monthly_sale['store_id'] == "CA_1") | (monthly_sale['store_id'] == "CA_2") | (monthly_sale['store_id'] == "CA_3") | (monthly_sale['store_id'] == "CA_4")) & (monthly_sale['item_id'] == "HOBBIES_1_001")].iloc[:,6:]

In [638]:
same_state_sales

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,...,month59,month60,month61,month62,month63,month64,month65,month66,month67,month68
0,0,0,0,0,0,0,0,0,0,0,...,22,18,16,12,21,26,19,30,28,9
3049,0,0,0,0,0,0,0,0,0,0,...,18,22,16,21,17,14,19,21,19,7
6098,0,0,0,0,0,0,0,0,0,0,...,18,11,23,27,24,20,22,27,30,9
9147,0,0,0,0,0,0,0,0,0,0,...,11,19,25,22,22,26,21,24,23,3


In [639]:
same_state_sales=same_state_sales.sum(axis=1)
same_state_sales

0       598
3049    538
6098    752
9147    632
dtype: int64

In [388]:
sell_prices.loc[((sell_prices["store_id"]=="CA_1") | (sell_prices["store_id"]=="CA_2") | (sell_prices["store_id"]=="CA_3") | (sell_prices["store_id"]=="CA_4")) & (sell_prices["item_id"]=="HOBBIES_1_001")]

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
2029946,CA_4,HOBBIES_1_001,11617,8.38
2029947,CA_4,HOBBIES_1_001,11618,8.38
2029948,CA_4,HOBBIES_1_001,11619,8.38
2029949,CA_4,HOBBIES_1_001,11620,8.38


In [467]:
ca_2=sell_prices.loc[(sell_prices["store_id"]=="CA_2") & (sell_prices["item_id"]=="HOBBIES_1_001")]['sell_price']

In [458]:
ca_1=sell_prices.loc[(sell_prices["store_id"]=="CA_1") & (sell_prices["item_id"]=="HOBBIES_1_001")]

In [459]:
ca_1

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38
150,CA_1,HOBBIES_1_001,11618,8.38
151,CA_1,HOBBIES_1_001,11619,8.38
152,CA_1,HOBBIES_1_001,11620,8.38


In [462]:
ca_2

,store_id,item_id,wm_yr_wk,sell_price
698412,CA_2,HOBBIES_1_001,11327,8.26
698413,CA_2,HOBBIES_1_001,11328,8.26
698414,CA_2,HOBBIES_1_001,11329,8.26
698415,CA_2,HOBBIES_1_001,11330,8.26
698416,CA_2,HOBBIES_1_001,11331,8.26
...,...,...,...,...
698559,CA_2,HOBBIES_1_001,11617,8.38
698560,CA_2,HOBBIES_1_001,11618,8.38
698561,CA_2,HOBBIES_1_001,11619,8.38
698562,CA_2,HOBBIES_1_001,11620,8.38


In [468]:
ca_1.reset_index(drop=True,inplace=True)

In [472]:
ca_2.reset_index(drop=True,inplace=True)

In [473]:
ca_12=pd.concat([ca_1,pd.DataFrame(ca_2)],axis=1,ignore_index=True)

In [474]:
ca_12

,0,1,2,3,4
0,CA_1,HOBBIES_1_001,11325,9.58,8.26
1,CA_1,HOBBIES_1_001,11326,9.58,8.26
2,CA_1,HOBBIES_1_001,11327,8.26,8.26
3,CA_1,HOBBIES_1_001,11328,8.26,8.26
4,CA_1,HOBBIES_1_001,11329,8.26,8.26
...,...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38,8.38
150,CA_1,HOBBIES_1_001,11618,8.38,8.38
151,CA_1,HOBBIES_1_001,11619,8.38,8.38
152,CA_1,HOBBIES_1_001,11620,8.38,NaN


In [475]:
ca_3=sell_prices.loc[(sell_prices["store_id"]=="CA_3") & (sell_prices["item_id"]=="HOBBIES_1_001")]['sell_price']

In [476]:
ca_4=sell_prices.loc[(sell_prices["store_id"]=="CA_4") & (sell_prices["item_id"]=="HOBBIES_1_001")]['sell_price']

In [478]:
ca_3.reset_index(drop=True,inplace=True)
ca_4.reset_index(drop=True,inplace=True)

In [517]:
ca_1234=pd.concat([ca_12,ca_3,ca_4],axis=1,ignore_index=True)

In [521]:
ca_1234=ca_1234.fillna(method='ffill')

In [522]:
ca_1234

,0,1,2,3,4,5,6
0,CA_1,HOBBIES_1_001,11325,9.58,8.26,8.26,9.58
1,CA_1,HOBBIES_1_001,11326,9.58,8.26,8.26,9.58
2,CA_1,HOBBIES_1_001,11327,8.26,8.26,8.26,9.58
3,CA_1,HOBBIES_1_001,11328,8.26,8.26,8.26,8.26
4,CA_1,HOBBIES_1_001,11329,8.26,8.26,8.26,8.26
...,...,...,...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38,8.38,8.38,8.38
150,CA_1,HOBBIES_1_001,11618,8.38,8.38,8.38,8.38
151,CA_1,HOBBIES_1_001,11619,8.38,8.38,8.38,8.38
152,CA_1,HOBBIES_1_001,11620,8.38,8.38,8.38,8.38


In [525]:
ca_1234['Average']=ca_1234[[3,4,5,6]].sum(axis=1)/4

In [526]:
ca_1234

,0,1,2,3,4,5,6,Average
0,CA_1,HOBBIES_1_001,11325,9.58,8.26,8.26,9.58,8.92
1,CA_1,HOBBIES_1_001,11326,9.58,8.26,8.26,9.58,8.92
2,CA_1,HOBBIES_1_001,11327,8.26,8.26,8.26,9.58,8.59
3,CA_1,HOBBIES_1_001,11328,8.26,8.26,8.26,8.26,8.26
4,CA_1,HOBBIES_1_001,11329,8.26,8.26,8.26,8.26,8.26
...,...,...,...,...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38,8.38,8.38,8.38,8.38
150,CA_1,HOBBIES_1_001,11618,8.38,8.38,8.38,8.38,8.38
151,CA_1,HOBBIES_1_001,11619,8.38,8.38,8.38,8.38,8.38
152,CA_1,HOBBIES_1_001,11620,8.38,8.38,8.38,8.38,8.38


In [536]:
y=ca_1234.iloc[:,-1]

In [539]:
y=y[:-8]

In [554]:
y

0      8.92
1      8.92
2      8.59
3      8.26
4      8.26
       ... 
141    8.29
142    8.32
143    8.38
144    8.38
145    8.38
Name: Average, Length: 146, dtype: float64

In [557]:
sum(y[8:12])

33.04

In [556]:
y[4:8]

4    8.26
5    8.26
6    8.26
7    8.26
Name: Average, dtype: float64

In [566]:
yy=[]
for i in range(0,len(y)-2,4):
    
    x=sum(y[i:i+4])/4

    yy.append(x)


In [567]:
yy.append(sum(y[-2:])/2)

In [568]:
yy

[8.6725,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.3125,
 8.38]

In [569]:
len(yy)

37

In [614]:
x=same_state_sales[31:].tolist()

In [615]:
x

[25,
 60,
 66,
 75,
 82,
 82,
 67,
 62,
 75,
 62,
 56,
 57,
 64,
 68,
 53,
 52,
 60,
 71,
 67,
 65,
 56,
 67,
 73,
 70,
 29,
 88,
 86,
 69,
 70,
 80,
 82,
 84,
 86,
 81,
 102,
 100,
 28]

In [616]:
x.reset_index(drop=True,inplace=True)

AttributeError: 'list' object has no attribute 'reset_index'

In [617]:
len(x)

37

In [543]:
sum(y[0:4])

34.69

In [618]:
x= sm.add_constant(x)

In [596]:
len(yy)

37

In [619]:
sm.OLS(yy,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     10.54
Date:                Wed, 13 Mar 2024   Prob (F-statistic):            0.00257
Time:                        18:45:58   Log-Likelihood:                 51.097
No. Observations:                  37   AIC:                            -98.19
Df Residuals:                      35   BIC:                            -94.97
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.4100      0.043    197.470      0.000       8.324       8.496
x1            -0.0020      0.001     -3.247      0.003      -0.003      -0.001
==============================================================================
Omnibus:                       59.332   Durbin-Watson:                   1.135
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              478.458
Skew:                           3.619   Prob(JB):                    1.27e-104
Kurtosis:                      19.061   Cond. No.                         291.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [623]:
#Try all state for one product

In [649]:
all_state_sales=monthly_sale.loc[(( monthly_sale['store_id'].str.contains("CA")  ) | (monthly_sale['store_id'].str.contains("WI")  ) | ( monthly_sale['store_id'].str.contains("TX") ) ) & (monthly_sale['item_id'] == "HOBBIES_1_001")].iloc[:,6:]

In [650]:
all_state_sales

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,...,month59,month60,month61,month62,month63,month64,month65,month66,month67,month68
0,0,0,0,0,0,0,0,0,0,0,...,22,18,16,12,21,26,19,30,28,9
3049,0,0,0,0,0,0,0,0,0,0,...,18,22,16,21,17,14,19,21,19,7
6098,0,0,0,0,0,0,0,0,0,0,...,18,11,23,27,24,20,22,27,30,9
9147,0,0,0,0,0,0,0,0,0,0,...,11,19,25,22,22,26,21,24,23,3
12196,0,0,0,0,0,0,0,0,0,0,...,5,7,8,5,6,4,6,11,9,2
15245,0,0,0,0,0,0,0,0,0,0,...,9,18,16,20,17,19,18,6,11,1
18294,0,0,0,0,0,0,0,0,0,0,...,14,17,14,23,14,19,17,13,10,3
21343,0,0,0,0,0,0,0,0,0,0,...,5,12,13,17,10,11,16,9,11,3
24392,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,6,1
27441,0,0,0,0,0,0,0,0,0,0,...,7,11,10,8,2,7,8,8,4,1


In [651]:
all_state_sales=all_state_sales.sum(axis=0)


month1       0
month2       0
month3       0
month4       0
month5       0
          ... 
month64    150
month65    146
month66    149
month67    151
month68     39
Length: 68, dtype: int64

In [652]:
all_state_sales

month1       0
month2       0
month3       0
month4       0
month5       0
          ... 
month64    150
month65    146
month66    149
month67    151
month68     39
Length: 68, dtype: int64

In [712]:
ca_1=sell_prices.loc[(sell_prices["item_id"]=="HOBBIES_1_001")]

In [713]:
ca_1

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6145173,WI_3,HOBBIES_1_001,11617,8.38
6145174,WI_3,HOBBIES_1_001,11618,8.38
6145175,WI_3,HOBBIES_1_001,11619,8.38
6145176,WI_3,HOBBIES_1_001,11620,8.38


In [657]:
ca_2=sell_prices.loc[(sell_prices["store_id"]=="CA_2") & (sell_prices["item_id"]=="HOBBIES_1_001")]

In [659]:
ca_12=pd.concat([ca_1,ca_2])

In [714]:
ca_12=ca_1

In [715]:
price=[]
wm_yr=ca_12['wm_yr_wk'][0]
for i in range(len(ca_12)):
    wm_yr=ca_12['wm_yr_wk'][i]
    if wm_yr==11621:
        data=ca_12[ca_12['wm_yr_wk']==wm_yr]['sell_price']
        count=len(data)
        price.append(sum(data)/count)
        
        
        break
    else:
        data=ca_12[ca_12['wm_yr_wk']==wm_yr]['sell_price']
        count=len(data)
        price.append(sum(data)/count)
        

In [741]:
price

154

In [717]:
week_available=len(price)-8

In [726]:
months=week_available//4

In [724]:
extra_week=week_available%4

In [727]:
if extra_week>0:
    months+=1

In [738]:
months

37

In [750]:
all_state_sales=all_state_sales[-months:]
all_state_sales

month32     48
month33     84
month34     92
month35    109
month36    127
month37    117
month38     96
month39     90
month40    113
month41     94
month42     92
month43     88
month44    101
month45    119
month46     84
month47     76
month48    108
month49     99
month50    111
month51    104
month52    107
month53    103
month54    100
month55    136
month56     83
month57    149
month58    135
month59    109
month60    135
month61    141
month62    155
month63    133
month64    150
month65    146
month66    149
month67    151
month68     39
dtype: int64

In [746]:
y=[]
if extra_week:
    for i in range(0,months*4-extra_week,4):
        data=sum(price[i:i+4])/4

        y.append(data)


In [751]:
y

[8.969,
 8.431000000000001,
 8.332,
 8.332,
 8.314,
 8.308000000000002,
 8.308000000000002,
 8.308000000000002,
 8.281000000000002,
 8.281,
 8.284,
 8.284,
 8.284,
 8.281,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.1905,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.272,
 8.287,
 8.343999999999998]

In [753]:
x=all_state_sales.values.tolist()

In [755]:
x

[48,
 84,
 92,
 109,
 127,
 117,
 96,
 90,
 113,
 94,
 92,
 88,
 101,
 119,
 84,
 76,
 108,
 99,
 111,
 104,
 107,
 103,
 100,
 136,
 83,
 149,
 135,
 109,
 135,
 141,
 155,
 133,
 150,
 146,
 149,
 151,
 39]

In [756]:
x= sm.add_constant(x)

In [757]:
sm.OLS(yy,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     9.041
Date:                Wed, 13 Mar 2024   Prob (F-statistic):            0.00486
Time:                        21:40:14   Log-Likelihood:                 50.476
No. Observations:                  37   AIC:                            -96.95
Df Residuals:                      35   BIC:                            -93.73
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.4016      0.043    194.845      0.000       8.314       8.489
x1            -0.0011      0.000     -3.007      0.005      -0.002      -0.000
==============================================================================
Omnibus:                       63.629   Durbin-Watson:                   1.172
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              593.388
Skew:                           3.935   Prob(JB):                    1.40e-129
Kurtosis:                      20.971   Cond. No.                         468.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""